In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies =  pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies.info()

In [ ]:
credits.info()

# Data Preprocessing

In [ ]:
movies = movies.merge(credits, on='title')

In [ ]:
movies.shape

In [ ]:
# movies['original_language'].value_counts()
# english movies are almost 95% so it wont help in analysis

In [ ]:
# cols that we are keeping 
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
# genres

movies.iloc[0].genres

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ['Action', 'Adventure', 'Fantasy', 'SciFi']

# objects are in string
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
# keywords
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
# cast - need top3 actors only
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

In [ ]:
# crew - we need only director
# we need the dictionary that has 'job = director'
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
movies['cast'][0]

In [ ]:
# overview is in 'string' we need it in a 'list'
# movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

In [ ]:
# Sam Worthington -> SamWorthington
# Science Fiction -> ScienceFiction

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head()

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new.head()

In [ ]:
# tags is list -> string
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
new['tags'][0]

In [ ]:
# lowercase
new['tags'] = new['tags'].apply(lambda x: x.lower())
new.head()

# Vectorization

In [ ]:
!pip install nltk

In [ ]:
# Stemming

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return ' '.join(y)

In [ ]:
new['tags'] = new['tags'].apply(stem)

In [ ]:
new.iloc[0].tags

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
similarity[0]

In [ ]:
# tuples (index, similarity)
list(enumerate(similarity[0]))

In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:11]:
        print(new.iloc[i[0]].title)
        

In [ ]:
recommend('Batman Begins')

In [ ]:
recommend('Gandhi')

In [ ]:
import pickle

In [ ]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
pickle.dump(new.to_dict(), open('movie_dict.pkl','wb'))

In [ ]:
new.to_dict()